In [1]:
library(Seurat)
library(dplyr)
seurat = readRDS('../initialseurat.RDS')
hashdata = read.csv('citeseq_results_dense.csv', header = 1, stringsAsFactors = FALSE, row.names = 1, sep = ',') #%>% t()
head(hashdata)

Warning message:
"package 'Seurat' was built under R version 4.1.3"
Attaching SeuratObject

Attaching sp

Warning message:
"package 'dplyr' was built under R version 4.1.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




,HTO1.ACCCACCAGTAAGAC,HTO2.GGTCGAGAGCATTCA,HTO3.CTTGCCGCATGTCAT,HTO4.AAAGCATTCTTCACG,HTO5.CTTTGTCTTTGTGAG,HTO6.TATGCTGCCACGGTA,HTO7.GAGTCTGCCAGTATC,HTO8.TATAGAACGCCAGGC,HTO9.TGCCTATGAAACAAG,HTO10.CCGATTGTAACAGAC,unmapped
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
R1CGGAATTTATCG,0,0,0,0,0,79,110,111,42,40,44
R1TGTTGAAGATAG,0,0,0,0,0,3,5,333,5,2,82
R1CTTGCCCTAAAA,0,0,0,0,0,54,75,329,28,28,89
R1CCCAGCCAGTCG,0,0,0,0,0,3,27,204,7,2,70
R1AATGCATTATCC,0,0,0,0,0,128,214,148,51,62,78
R1GAGGCCCGGTCG,0,0,0,0,0,11,13,21,12,213,70


In [2]:
metaex = read.table('../LMCarT_meta.txt', sep = ',', header = 1, row.names =1 )

In [3]:
head(metaex)

,orig,bc,n_genes,n_counts,leiden,id,Treatment,TimePoint
,<chr>,<chr>,<int>,<dbl>,<int>,<chr>,<chr>,<chr>
Q1_0,Q1,AAAAAAAGCCCC,1730,3428,4,Q1AAAAAAAGCCCC,NoVax,7day
Q1_1,Q1,AAAAAAGATAGA,841,2090,0,Q1AAAAAAGATAGA,NoVax,7day
Q1_2,Q1,AAAAACTCACGC,3914,12815,3,Q1AAAAACTCACGC,NoVax,7day
Q1_3,Q1,AAAAAGATACTA,1080,1844,3,Q1AAAAAGATACTA,NoVax,7day
Q1_4,Q1,AAAAAGATTACC,919,3151,3,Q1AAAAAGATTACC,NoVax,7day
Q1_5,Q1,AAAAAGCAAGTA,1879,4458,1,Q1AAAAAGCAAGTA,NoVax,7day


In [4]:
seurat

An old seurat object
 17039 genes across 53901 samples

In [5]:
seurat = UpdateSeuratObject(seurat)

Updating from v2.X to v3.X

Validating object structure

Updating object slots

Ensuring keys are in the proper strucutre

Ensuring feature names don't have underscores or pipes

Object representation is consistent with the most current Seurat version



In [6]:
seurat$id = paste0(seurat$orig,  seurat$bc)
head(seurat$id)
seurat = subset(seurat, cell = colnames(seurat)[seurat$id %in% rownames(hashdata)])
seurat

Q1_0             Q1_1             Q1_2             Q1_3 
"Q1AAAAAAAGCCCC" "Q1AAAAAAGATAGA" "Q1AAAAACTCACGC" "Q1AAAAAGATACTA" 
            Q1_4             Q1_5 
"Q1AAAAAGATTACC" "Q1AAAAAGCAAGTA"

An object of class Seurat 
17039 features across 53650 samples within 1 assay 
Active assay: RNA (17039 features, 220 variable features)
 2 dimensional reductions calculated: pca, umap

In [7]:
hash_mat = hashdata[seurat$id, 1:10]
bcs_to_names = colnames(seurat)
names(bcs_to_names) = seurat$id
rownames(hash_mat) = bcs_to_names[rownames(hash_mat)]

In [8]:
seurat[['HTO']] = CreateAssayObject(counts = t(hash_mat))
#seurat = subset(seurat, cells =colnames(seurat)[seurat$n_genes > 1000])

In [9]:
seurat = subset(seurat, cells = colnames(seurat)[seurat@meta.data$nCount_HTO > 30])

In [10]:
library(MASS)
library(cluster)
library(outliers)

#seurat$hash = NA
Idents(seurat) = 'orig'
for (curr in unique(seurat$orig)) {
    message(curr)
    sub =subset(seurat, idents =  curr)
    
    if (length(colnames(sub)) > 50) {
        if (curr %in%  c('Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10'))  {
        sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),6:10]))
            }
    
        sub = NormalizeData(sub, assay = 'HTO', normalization.method = 'CLR', margin = 2)

        DefaultAssay(sub) = 'HTO'
        data = GetAssayData(sub)


        # k medioids
        meds = pam(t(data), k = dim(data)[1])
        calls = t(data)
        cluster_data = meds$medoids %>% as.matrix()
        

        for (i in 1:dim(cluster_data)[2]) {
            
  

            curr_med = which.max(cluster_data[,i])
            negs = meds$clustering[meds$clustering != curr_med]
            neg_vals = data[i, names(negs)]
            pos_vals = data[i, meds$clustering == curr_med]
            pos_vals = pos_vals[order(pos_vals)] 
            done = FALSE

            # remove outliers and set threshold
            while(!done & length(pos_vals) > 5) {
                done = TRUE
                avg = mean(pos_vals)
                diff1 = avg - min(pos_vals)
                diff2 = max(pos_vals) - avg
                test = grubbs.test(pos_vals, opposite = diff1 < diff2)
                if (test$p.value < 0.05) {
                    pos_vals = pos_vals[-1]
                    done = FALSE
                }
                thresh = min(pos_vals)

            }
            print(curr_med)
            print(thresh)
            if (thresh < .3) {
                thresh = 1.5
                message('danger')
            }
            
            if (curr == 'R7') {
                thresh = 3
            }
            if (curr == 'R8') {
                thresh = 2
            }
            calls[,i] = as.numeric(calls[,i] >= thresh)
            assign = apply(calls, 1, which.max)
            
            

            assign = paste0( 'HTO', assign + 5)
            rsums = rowSums(calls)
            
        
            
            assign[rsums > 1] = 'Doublet'
            assign[rsums == 0] = 'Negative'
 
            seurat@meta.data[colnames(sub), 'hash'] = assign
            pheatmap::pheatmap(t(data), annotation_row = data.frame(row.names = colnames(data), id = factor(assign)), show_rownames = FALSE)

            pheatmap::pheatmap(t(data), annotation_row = data.frame(row.names = colnames(data), id = factor(assign)), filename = paste0(curr, '.png'), show_rownames = FALSE)

        }
    }
}


Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select


Warning message:
"package 'outliers' was built under R version 4.1.1"
Q1

Normalizing across cells



Q1_2974 
      3 
[1] 0.6137013
Q1_2836 
      5 
[1] 1.285556
Q1_1060 
      2 
[1] 1.544092
Q1_2861 
      4 
[1] 1.162945
Q1_3854 
      1 
[1] 0.7377633


Q2

Normalizing across cells



Q2_5758 
      5 
[1] 0.6818403
Q2_6318 
      4 
[1] 0.8820031
Q2_4448 
      3 
[1] 0.7930894
Q2_34 
    1 
[1] 0.9033631
Q2_6014 
      2 
[1] 0.6135518


Q3

Normalizing across cells



...

R9_3159 
      5 
[1] 1.886548
R9_2653 
      1 
[1] 1.884057
R9_1946 
      4 
[1] 2.046992
R9_3709 
      2 
[1] 1.477325


R10

Normalizing across cells



R10_1303 
       1 
[1] 0


danger



R10_2623 
       4 
[1] 1.727239
R10_3763 
       2 
[1] 1.72209
R10_2189 
       3 
[1] 2.035441
R10_1564 
       5 
[1] 1.437094


In [11]:
table(seurat@meta.data$hash)


 Doublet    HTO10     HTO6     HTO7     HTO8     HTO9 Negative 
    6564     8860     5880     9337     8267     5870     8793 

In [12]:
saveRDS(seurat@meta.data, 'hashed_meta.RDS')